**Order Agent**

In [ ]:
import json
import os
import uuid
from datetime import datetime

import pytz
import requests
from flask import Flask, request, jsonify
import re
from config import Config
from groclake.modellake import Modellake
from groclake.datalake import Datalake
from groclake.utillake import GrocAgent

from groclake.memorylake import Memorylake

# Initialize Flask app
app = Flask(__name__)

model = Modellake()
memory_lake = Memorylake()


class DatalakeConnection(Datalake):
    def __init__(self):
        super().__init__()

        MYSQL_CONFIG = Config.MYSQL_CONFIG
        MYSQL_CONFIG['connection_type'] = 'sql'

        self.plotch_pipeline = self.create_pipeline(name="groclake_pipeline")
        self.plotch_pipeline.add_connection(name="sql_connection", config=MYSQL_CONFIG)

        self.execute_all()

        self.connections = {
            "sql_connection": self.get_connection("sql_connection")
        }

    def get_connection(self, connection_name):
        """
        Returns a connection by name from the pipeline.
        """
        return self.plotch_pipeline.get_connection_by_name(connection_name)


datalake_connection = DatalakeConnection()
mysql_connection = datalake_connection.connections["sql_connection"]


class CraftsvillaOrderAgent(GrocAgent):
    def __init__(self, app, agent_name, initial_intent=None, intent_description=None, adaptor_config=None):
        self.intent_handler = self.default_handler
        super().__init__(app, agent_name, initial_intent, intent_description, self.intent_handler, adaptor_config)
        self.sql_query = ''
        self.registerHandler('ecommerce_order_cancel', self.ecommerce_order_cancel_handler)

    def execute_query(self, query, params=None):
        """Execute a SQL query and return results."""
        try:
            sql_conn = mysql_connection
            if query.lower().startswith("select"):
                return sql_conn.read(query, params=params, multiple=True)
            return {}

        except Exception as err:
            print('db error', err)
            return {}

    # memory functions
    def get_store_memory(self, customer_id, n=1):
        user_id = "00a2a625-24b6-4a53-8a38-da49c0cb21b9"
        memory_context = {
            "context_entity_id": customer_id,  # guest ,email,customer_no,vendor_id
            "context_id": "00a2a625-24b6-4a53-8a38-da49c0cb21b9",
            "memory_type": 1,
            # "memory_id": "msg10"
        }
        conversations = memory_lake.short_memory_read(user_id, memory_context, n=n)
        return conversations

    def get_current_datetime(self):
        return datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S")

    def create_memory_for_context_completion(self, query_text, context_entities, response, metadata, customer_id):
        user_id = "00a2a625-24b6-4a53-8a38-da49c0cb21b9"

        memory_context = {
            "context_entity_id": f"customer:{customer_id}",  # guest ,email,customer_no,vendor_id
            "context_id": "00a2a625-24b6-4a53-8a38-da49c0cb21b9",
            "memory_type": 1,
            "memory_id": str(uuid.uuid4()),
        }

        memory = {
            "query_text": query_text,
            "response_text": response,
            "time": self.get_current_datetime(),
            "intent": "order_status",
            "entities": context_entities,
            "metadata": metadata,
            "cache_ttl": 3600
        }

        try:
            result = memory_lake.short_memory_create(user_id, memory_context, memory)
            if not result:
                print("Memory storage failed!")
        except ValueError as e:
            print(f"Memory Error: {e}")

    def get_store_memory_for_context_completion(self, customer_id):
        user_id = "00a2a625-24b6-4a53-8a38-da49c0cb21b9"
        memory_context = {
            "context_entity_id": f"customer:{customer_id}",
            "context_id": "00a2a625-24b6-4a53-8a38-da49c0cb21b9",
            "memory_type": 1
        }
        conversations = memory_lake.short_memory_read(user_id, memory_context, n=1)
        if isinstance(conversations, dict):
            filtered_results = {
                key: value
                for key, value in conversations.items()
                if customer_id in key
            }
            return filtered_results
        return conversations

    def create_memory(self, query_text, response, intent, entities, customer_id, metadata):
        user_id = "00a2a625-24b6-4a53-8a38-da49c0cb21b9"
        memory_context = {
            "context_entity_id": customer_id,  # guest ,email,customer_no,vendor_id
            "context_id": "00a2a625-24b6-4a53-8a38-da49c0cb21b9",
            "memory_type": 1,
            "memory_id": str(uuid.uuid4())
        }

        memory = {
            "query_text": query_text,
            "response_text": response,
            "time": self.get_current_datetime(),
            "intent": intent,
            "entities": entities,
            "metadata": metadata,
            "cache_ttl": 3600

        }

        try:
            result = memory_lake.short_memory_create(user_id, memory_context, memory)
            if not result:
                print("Memory storage failed!")
        except ValueError as e:
            print(f"Memory Error: {e}")

    # End memory functions

    # get last stored order id from context memory
    def get_stored_order_id_from_context(self, customer_id):
        order_id = ""
        stored_order_id = self.get_store_memory_for_context_completion(customer_id)
        if isinstance(stored_order_id, dict):
            for key, value in stored_order_id.items():
                entities = value.get("entities")
                if entities and isinstance(entities, dict):  # Check if entities exist and is a dictionary
                    order_id = entities.get("order_id", "")
            return order_id

    # get Past conversation
    def get_past_conversation(self, customer_id, n=1):
        past_conversation_text = ""
        stored_memory_conversations = self.get_store_memory(customer_id, n=n)
        if isinstance(stored_memory_conversations, dict):
            for key, value in stored_memory_conversations.items():
                query = value.get("query_text", "")
                response_text = value.get("response_text", "")
                past_conversation_text += f"user query : {query} -> Response to user query: {response_text} || "
        return past_conversation_text

    def rewrite_query_text(self, query_text, customer_id, agent_details, n=2):
        # Agent description
        agent_name = agent_details.get('agent_name', '')
        agent_description = agent_details.get('agent_description', '')
        agent_role = agent_details.get('agent_role', '')

        context_attributes = {}
        # Get stored order_id from context memory
        order_id = self.get_stored_order_id_from_context(customer_id)
        context_attributes.update({"order_id": order_id})
        # Get stored past conversation
        # past_conversation_text = self.get_past_conversation(customer_id, n=n)
        past_conversation_text = ''
        stored_memory_conversations = self.get_store_memory(customer_id, n=n)
        if isinstance(stored_memory_conversations, dict):
            for key, value in stored_memory_conversations.items():
                query = value.get("query_text", "")
                past_conversation_text += f"user query : {query}"

        # is_another_order_case = any(phrase in query_text.lower() for phrase in [
        #     "another order", "different order", "other order", "one more order", "another ticket", "other order",
        #     "different order"
        # ])

        # context_attributes = {}

        # # Only include order_id if not asking about another order
        # if is_another_order_case and not order_id:
        #     context_attributes.update({"order_id": ''})
        #     past_conversation_text = ''

        # Set up agent config
        agent_config = {
            "query_text": query_text,
            "agent_last_conversation": past_conversation_text,
            "agent_name": agent_name,
            "agent_role": agent_role,
            "agent_description": agent_description,
            "context_attributes": context_attributes if context_attributes else ""
        }
        rewrite_query_text = self.rewrite_query(agent_config)
        return rewrite_query_text

    def extract_sql_query(self, text):
        sql_query = re.search(r'```sql\n(.*?)\n```', text, re.DOTALL)
        if sql_query:
            extracted_query = sql_query.group(1)
            return extracted_query
        else:
            return {}

    def query_entity_extraction(self, query_text, extract_data):
        system_prompt = f"""
                    Analyze the given user query along with past conversations to extract the specified fields.

                    ### Extraction Rules:
                    current query_text : {query_text}
                    - If the query contains values for any of these fields: {extract_data}, extract and return them.
                    - If any field value is missing in the current query, retrieve it from the past conversation if available.
                    - If a user provides a number (e.g., "57789"), store it as the "order_id" in response data.
                    - If all requested fields are found, return them in JSON format.
                    - If no relevant information is found in the query or past memory, return an **empty JSON object**.
                    - **Do not invent or assume values** if they are not explicitly present.

                     ### Example Scenarios:
                    - **Input Query:** "What is the status of order?"
                      **Output:** `{{ "order_id": "" }}`
                    - **Input Query:** "98765"
                      **Output:** `{{ "order_id": "98765" }}`
                    - **Input Query:** "Check customer 54321's last order."
                      **Output:** `{{ "customer_id": "54321" }}`
                    - **Input Query:** "Can you help me?" (No relevant data)
                      **Output:** `{{}}`

                    ### Expected Output (JSON format):
                                ```json
                                {json.dumps(extract_data, indent=4)}
                                ```
                    """
        response_payload = {
            "messages": [
                {"role": "system", "content": system_prompt},
                # {"role": "user", "content": user_prompt}
            ]
        }
        try:
            response1 = model.chat_complete(payload=response_payload)
            response_text = response1.get('answer', "Unable to generate a response.")
            if "```json" in response_text:
                start_index = response_text.find("```json") + 7
                end_index = response_text.rfind("```")
                extracted_json = response_text[start_index:end_index].strip()
            else:
                extracted_json = response_text.strip()
            valid_data = json.loads(extracted_json)
            if isinstance(valid_data, dict):
                if len(valid_data.get('order_id')) <= 2:
                    valid_data['order_id'] = ''
            return valid_data
        except Exception as e:
            return extract_data

    def validate_order_id(self, order_id, customer_id):
        valid_order_id = self.get_valid_order_id(order_id, customer_id).get('valid_order_id')
        if not valid_order_id:
            return "please provide valid numeric order ID."
        query = f"select customer_id from retail_customer where mobile={customer_id}  and customer_instance = {Config.CUSTOMER_INSTANCE} limit 1;"
        customer_id = None
        mobile_number = self.execute_query(query)
        if mobile_number:
            customer_id = mobile_number[0].get('customer_id', '')
        query = f"select customer_id,order_id from retail_sales where customer_id = {customer_id} and order_id = {valid_order_id} limit 1;"
        result = self.execute_query(query)
        return result

    def get_valid_order_id(self, order_id, customer_id):
        past_execution_result = ""
        past_order_id = self.get_stored_order_id_from_context(customer_id)
        valid_order_id = None
        if order_id and order_id.isdigit():
            valid_order_id = order_id
        elif past_order_id and past_order_id.isdigit():
            valid_order_id = past_order_id
        return {'valid_order_id': valid_order_id, 'past_order_id': past_order_id,
                'past_execution_result': past_execution_result}

    def get_customer_id_from_retail_sales(self, customer_id):
        query = f"select customer_id from retail_customer where mobile={customer_id}  and customer_instance = {Config.CUSTOMER_INSTANCE} limit 1;"
        customer_id = None
        mobile_number = self.execute_query(query)
        if mobile_number:
            customer_id = mobile_number[0].get('customer_id', '')
        return customer_id

    def extract_item_name_from_query(self, customer_id, query_text):
        query = ""
        stored_memory_conversations = self.get_store_memory(customer_id, n=1)
        if isinstance(stored_memory_conversations, dict):
            for key, value in stored_memory_conversations.items():
                query = value.get("query_text", "")
        system_prompt = """You are an intelligent AI assistant designed to extract product names from user queries related to order cancellations.

                               You are an intelligent AI assistant designed to extract product names from user queries related to order cancellations.

                                ### Task:
                                - Identify and return the **item name** mentioned in the user's query.
                                - If the user does **not** specify a product/item name, return `"item_name": ""`.
                                - If the user gives only an **order ID** or refers to an "order" without mentioning an item, do **not** treat that as an item name.

                                ### Rules:
                                - Do NOT extract anything that includes:
                                  - phrases like "order", "order ID", "order number", "thing", "product", "item", "stuff"
                                  - any text containing digits (e.g., "55814740", "order 12345")
                                - Only extract **clear product names** like "iPhone", "Adidas shoes", "MacBook", etc.
                                - If the item name is **not found in current query**, check the most recent past query provided.

                                ### Expected Output:
                                - Return a **JSON object** containing the extracted **item name**.

                                ### Example Queries & JSON Responses:

                                #### User: "I want to cancel my order for the iPhone 14 Pro."
                                **Response:**
                                ```json{  "item_name": "iPhone 14 Pro"}
                                ```
                                #### User: "Cancel my order for the Adidas shoes."
                                **Response:**
                                ```json { "item_name": "Adidas shoes" }
                                ```
                                #### User: "I need to return my MacBook."
                                **Response:**
                                ```json { "item_name": "MacBook" }
                                ```
                                #### User: "Cancel my order." (No item name specified)
                                **Response:**
                                ```json { "item_name": ""}
                                ```
                                """
        user_prompt = f"""
                               ### **User Query Details**
                               - **Customer Query:** "{query_text}"
                               ### **Past user query:{query}"
                               """
        response_payload = {
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        }
        try:
            response1 = model.chat_complete(payload=response_payload)
            response_text = response1.get('answer', "Unable to generate a response.")
            if "```json" in response_text:
                start_index = response_text.find("```json") + 7
                end_index = response_text.rfind("```")
                extracted_json = response_text[start_index:end_index].strip()
            else:
                extracted_json = response_text.strip()
            valid_data = json.loads(extracted_json)
            return valid_data
        except Exception as e:
            return {"item_name": ""}

    def get_all_pending_orders(self, customer_id):
        retail_sales_customer_id = self.get_customer_id_from_retail_sales(customer_id)
        query = f"SELECT rsi.product_id, rsi.name, rsi.order_id, rsi.tracking_url, rsi.status, rsi.shipped_date, rsi.expected_delivery_date, rsi.created_at, rsi.qty FROM retail_sales_item rsi JOIN retail_sales rs ON rsi.order_id = rs.order_id WHERE rs.customer_id = {retail_sales_customer_id} AND rsi.status IN (0, 1)  ORDER BY rs.created_at DESC LIMIT 5;"
        result = self.execute_query(query)
        return result

    def get_order_by_order_id(self, order_id):
        query = f"SELECT order_item_id,product_id,name,order_id, status, tracking_url, expected_delivery_date ,created_at,qty FROM retail_sales_item WHERE order_id = {order_id};"
        result = self.execute_query(query)
        return result

    def get_products_from_khoj(self, product_name):
        default_response = [{
            'brand': '',
            'category_name': '',
            'client_item_id': '',
            'image_link1': '',
            'image_link2': '',
            'image_link3': '',
            'price': 0,
            'product_id': 0,
            'product_name': "",
            'product_name_hindi': None,
            'product_name_kannada': None,
            'product_name_tamil': None,
            'product_name_telugu': None,
            'product_url': '',
            'provider_id': '',
            'provider_name': '',
            'provider_name_hindi': None,
            'provider_name_kannada': None,
            'provider_name_tamil': None,
            'provider_name_telugu': None,
            'provider_url': '',
            'rating': '',
            'sale_price': 0,
            'score': 0,
            'weight': '0.0'
        }]

        payload = {
            "header": {
                "version": "1.0",
                "message": "Request",
                "Content-Type": "application/json",
                "apc_id": "c9d1c9b9-9f1b-4430-9bd4-6994dc1c89ee",
                "server_agent_uuid": "075fcb88-f25a-4390-a37e-e374a3c2b1df",
                "client_agent_uuid": "d0df97b5-711b-4d84-b9aa-891890410cfc",
                "message_id": "msg101",
                "task_id": "task1"
            },
            "body": {
                "query_text": product_name,
                "intent": "Search",
                "entities": [{
                    "image_bytes": "",
                    "image_url": "https://example.com/image.jpg",
                    "search_type": "semantic"
                }],
                "metadata": {
                    "context": "This search is happening on a khoj",
                    "cataloglake_id": "25ae360869404295",
                    "groc_account_id": "c4ca4238a0b923820dcc509a6f75849b",
                    "customer_id": "9980897430",
                    "session_agent_uuid": "aa",
                    "enhance_query": 0
                }
            }
        }

        endpoint = "https://api-uat-cartesian.groclake.ai/agache/agent/d0df97b5-711b-4d84-b9aa-891890410cfc/query"

        try:
            response = requests.post(endpoint, json=payload, timeout=5)
            response_data = response.json()
            return response_data.get('body', {}).get('entities', default_response)
        except requests.exceptions.RequestException as e:
            print(f"Error during API request: {e}")
            return default_response

    def order_response_entities(self, execution_result, intent=None):
        response_list = []
        response_entities = [{
            "Order_status": "",
            "Order_Id": "",
            "Quantity": "",
            "Order_Placed_on": "",
            "Product_name": "",
            "Product_expected_delivery_date": "",
            "Product_image": ""
        }]
        if execution_result:
            if intent == 'ecommerce_cancel_reason':
                khoj_result = self.get_products_from_khoj(execution_result[0].get('name'))
                if khoj_result:
                    response_list = khoj_result
                    return response_list
            for record in execution_result:
                product_id = record.get('product_id')
                query = f"""SELECT main_image FROM crs_products WHERE product_id= {product_id};"""
                crs_products_execution_result = self.execute_query(query)
                image_url = ''
                if crs_products_execution_result:
                    main_image = crs_products_execution_result[0].get('main_image', '')
                    if main_image is None or main_image == '':
                        image_url = ''
                    elif main_image.startswith('http'):
                        image_url = main_image
                    else:
                        image_url = f"https://cdnaz.plotch.io/image/upload{main_image}"
                response_entities = {
                    "Order_status": self.order_status_code().get(record.get('status', '')),
                    "Order_Id": record.get('order_id', ''),
                    "Quantity": record.get('qty', ''),
                    "Order_Placed_on": record.get('created_at', ''),
                    "Product_name": record.get('name', ''),
                    "Product_expected_delivery_date": record.get('expected_delivery_date', ''),
                    "Product_image": image_url,
                }
                response_list.append(response_entities)
        return response_list if response_list else response_entities

    def get_item_id_by_name(self, item_name, order_id):
        query = f"SELECT status,order_item_id,product_id,name,order_id FROM retail_sales_item WHERE name LIKE '{item_name[:10]}%' and order_id={order_id} LIMIT 1;"
        result = self.execute_query(query)
        return result

    def get_all_cancel_reason(self):
        reasons = [
            "Price of one or more items in the order have changed due to which buyer was asked to make additional payment",
            "One or more items in the order not available",
            "Product available at lower than order price",
            "Store is not accepting order",
            "Store rejected the order",
            "Order / fulfillment not received as per buyer app TAT SLA",
            "Order / fulfillment not received as per buyer app TAT SLA",
            "Order / fulfillment not ready for pickup",
            "Wrong product delivered",
            "Buyer wants to modify address / other order details",
            "Buyer not found or cannot be contacted",
            "Buyer does not want product any more",
            "Buyer refused to accept delivery",
            "Delivery address incorrect or not found",
            "Buyer not available at location",
            "Accident / rain / strike / vehicle issues",
            "Order delivery delayed or not possible due to damages, etc.",
            "Delivery address not serviceable",
            "Pickup pin code not serviceable",
            "Order lost in transit",
            "Order confirmation failure",
            "Order confirmation failure"
        ]
        return reasons

    def order_status_response_of_cancel_order_intent(self, order_id, customer_id, metadata, query_text, intent,
                                                     session_token, new_query_text):
        print('query text', query_text)
        agent_name = "Order Cancellation Assistant"
        agent_description = "an AI assistant specializing in order cancellations, refund processing, and issue resolution."
        agent_role = "assist users with canceling orders, processing refunds, and resolving cancellation-related queries."

        valid_order_details = self.get_valid_order_id(order_id, customer_id)
        valid_order_id = valid_order_details.get('valid_order_id')
        new_order_id = valid_order_id
        past_order_id = valid_order_details.get('past_order_id')
        past_conversation_text = self.get_past_conversation(customer_id, n=4)

        followup_text = ''
        item_name = self.extract_item_name_from_query(customer_id, query_text).get('item_name')
        cancel_api_response = ''
        response_entities_execution_result = []

        if intent == "ecommerce_another_order":
            if not new_order_id:
                past_conversation_text = ''
                query_text = new_query_text
                response_entities_execution_result = []
            if past_order_id and new_order_id == past_order_id:
                cancel_api_response = "Please provide the new Order ID you would like help with."
                response_entities_execution_result = self.get_all_pending_orders(customer_id)
                new_order_id = None
                item_name = ''
                past_conversation_text = ''
            else:
                cancel_api_response = f"You're referring to a different order. provide a new Order ID"

        if new_order_id:
            response_entities_execution_result = self.get_order_by_order_id(new_order_id)

            if not response_entities_execution_result:
                cancel_api_response = f"Could not find any details for Order ID {new_order_id}. Please check the Order ID."
            elif len(response_entities_execution_result) > 1:
                if not item_name:
                    cancel_api_response = "Your order contains multiple items. Please specify which one you'd like to cancel."
                else:
                    # First validate the item exists in the order
                    order_details = self.get_order_by_order_id(new_order_id)
                    item_name_lower = item_name.strip().lower()
                    all_item_names = [item.get('name', '').lower() for item in order_details]
                    matched_items = [name for name in all_item_names if name.strip().startswith(item_name_lower)]

                    if not matched_items:
                        cancel_api_response = f"The item '{item_name}' is not found in your order. Please provide a valid or more specific item name."
                    else:
                        # Find the first matching item (you could also handle multiple matches here)
                        matched_item_name = matched_items[0]
                        item_details = [item for item in order_details if
                                        item.get('name', '').lower().startswith(item_name_lower)]

                        if item_details:
                            order_item_id = item_details[0].get('order_item_id')
                            status = item_details[0].get('status')
                            product_id = item_details[0].get('product_id')
                            cancel_api_response, api_response = self.ecommerce_cancel_intiate(
                                new_order_id, customer_id, query_text, session_token, order_item_id, product_id, status
                            )
                            if api_response:
                                response_entities_execution_result = self.get_order_by_order_id(new_order_id)

            else:
                single_item = response_entities_execution_result[0]
                order_item_id = single_item.get('order_item_id')
                product_id = single_item.get('product_id')
                status = single_item.get('status')
                cancel_api_response, api_response = self.ecommerce_cancel_intiate(
                    order_id, customer_id, query_text, session_token, order_item_id, product_id, status
                )
                if api_response:
                    response_entities_execution_result = self.get_order_by_order_id(new_order_id)
        else:
            response_entities_execution_result = self.get_all_pending_orders(customer_id)
            if response_entities_execution_result:
                followup_text = f"Here are your pending orders ({len(response_entities_execution_result)}). Please provide an Order ID you'd like to cancel."
            else:
                followup_text = "You currently have no pending orders available for cancel."

        # Prepare structured prompt only if we have a clear order or instruction
        # system_prompt = f"""
        #        ## You are an AI-powered assistant named {agent_name}, {agent_description}.
        #        Your role is to {agent_role}. Handle user requests clearly and human-like.
        #
        #        Guidelines for response_text:
        #        - Greet warmly if greeting detected, ignore past conversation.
        #        - Always confirm the Order ID and guide for item name if needed.
        #        - cancellation reason must be given by user after order_id or item_name.
        #        - Never confirm or imply that the order has been cancelled (e.g., phrases like "your order cancellation has been initiated") unless the cancel API response explicitly confirms it.
        #        - If a cancel API response is provided, strictly use to it while constructing your reply but answer in user's query language.
        #        - Keep tone natural and avoid tech jargon.
        #
        #        ### **Follow-Up Text Instructions**
        #        - `follow_up_text` should provide **clear next steps**.
        #        - Keep it short, direct, and helpful.
        #        - If 'followup_text' provided use that in user query langauge.
        #        -  The follow_up_text must be included with every response without exception. It should always guide the user on the next steps.
        #
        #        ### **Language Rule**
        #        ### Key Notes:
        #        - **Strict language matching**: The `response_text` and `follow_up_text` should both be in the **same language** as the user's query.
        #        - **No mixing of languages**: If the user queries in Marathi, for example, don't mix it with Hindi or English in the response. The assistant should only respond in Marathi.
        #        - **Do not mix or switch languages** based on past conversations.
        #        - **Even if `cancel_api_response` or `past_conversation` is in a different language, always prioritize the language of the user's current query (`query_text`).
        #
        #
        #        Output format:
        #        Instruction:
        #        - output must be in **strict JSON format only**.
        #        - Do not include labels, markdown, or additional text outside the JSON.
        #        **Important:** Both `response_text` and `follow_up_text` **must be in the same language** as the user 'query_text'.
        #
        #        ```json
        #        {{
        #          "response_text": "Final message to customer with Order ID.",
        #          "follow_up_text": "Next steps or what user should do now."
        #        }}
        #        ```
        #        # Example for different languages Handle:
        #
        #        #User Query (English): "I want to cancel my order"
        #        ```json
        #        {{
        #           "response_text": "The cancellation process for your order ID 53159959 has been initiated.",
        #           "follow_up_text": "Please wait for a while, you will receive a confirmation message shortly."
        #        }} ```
        #
        #        # User Query (Hindi): "मुझे ऑर्डर ID 53159959 रद्द करना है।"
        #
        #        ```json
        #        {{
        #          "response_text": "आपका ऑर्डर ID 53159959 रद्द करने की प्रक्रिया शुरू कर दी गई है।",
        #          "follow_up_text": "कृपया कुछ समय प्रतीक्षा करें, आपको पुष्टि संदेश शीघ्र ही प्राप्त होगा।"
        #        }} ```
        #
        #         """

        language_prompt = f"""
            You are a language detection assistant. Detect the natural language of the following query.

            Instructions:
            - Return only the name of the detected language.
            - If the query is written in **pure Hindi (Devanagari script)**, return **Hindi**.
            - If the query is written in **pure Marathi (Devanagari script)**, return **Marathi**.
            - If the query is a **mix of Hindi and English written in Roman script**, return **Hinglish**.
              Examples of such code-mixed text: "mujhe red dress dikhao", "branded shoes chahiye", "mobile ka cover dikhao".

            Strict Rules:
            - Do not explain your answer.
            - Do not provide multiple options.
            - Only return a single word: the name of the language.

            Query: "{query_text}"
            """
        payload = {
            "messages": [
                {"role": "system", "content": language_prompt},
                {"role": "user", "content": query_text}
            ],
            "token_size": 100
        }
        response = model.chat_complete(payload=payload)
        detected_language = response.get("answer", "English").strip()

        system_prompt = f"""
           ## You are an AI-powered assistant named {agent_name}, {agent_description}.
           Your role is to {agent_role}. Handle user requests clearly and human-like.

           Guidelines for response_text:
           - Greet warmly if greeting detected, ignore past conversation.
           - Always confirm the Order ID and guide for item name if needed.
           - cancellation reason must be given by user after order_id or item_name.
           - Never confirm or imply that the order has been cancelled unless the cancel API response explicitly confirms it.
           - If a cancel API response is provided, strictly use it while constructing your reply but answer in user's query language.
           - Keep tone natural and avoid tech jargon.

           ### Follow-Up Text Instructions
           - follow_up_text should provide clear next steps.
           - Must always be in the same language as query_text.

           ### Language Rule
           - Respond in this language: **{detected_language}**.
           - Match the script strictly (e.g., Hinglish uses Roman script, Hindi uses Devanagari).
           - Do not mix languages or scripts.
           - **Important:** Both `response_text` and `follow_up_text` **must be in the same language** as the user 'query_text'.


           Output JSON only:
           {{
             "response_text": "Final message to customer with Order ID.",
             "follow_up_text": "{followup_text}"
           }}
           """
        user_prompt = f"""
               Customer Query: "{query_text}"
               Order ID: "{new_order_id}"
               Past Conversation: "{past_conversation_text}"
               Cancel API response : "{cancel_api_response}"
               Followup: "{followup_text}"
               Item Name: "{item_name}"
               Number of Items in Order: {len(response_entities_execution_result)}
               Order Details: {response_entities_execution_result if len(response_entities_execution_result) == 1 else ""}
               """

        response_payload = {
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            "model_name": "gpt-4o-mini"
        }

        response = model.chat_complete(payload=response_payload).get('answer')
        try:
            if "```json" in response:
                start_index = response.find("```json") + 7
                end_index = response.rfind("```")
                extracted_json = response[start_index:end_index].strip()
            else:
                extracted_json = response.strip()

            valid_data = json.loads(extracted_json)
            # if followup_text:
            #     valid_data['follow_up_text'] = followup_text
        except Exception:
            valid_data = {
                "response_text": response.get('response_text') if isinstance(response, dict) else response,
                "follow_up_text": "Let me know your Order ID or the item name you'd like to cancel."
            }

        # Save memory if needed
        if new_order_id and new_order_id.isdigit() and response:
            context_entities = {"order_id": new_order_id}
            self.create_memory_for_context_completion(query_text, context_entities, valid_data['response_text'],
                                                      metadata, customer_id)

        self.create_memory(query_text, valid_data['response_text'], intent, [], customer_id, metadata)

        response_entities = self.order_response_entities(response_entities_execution_result)
        sql_query = ''

        return valid_data, response_entities, sql_query

    def get_cancel_reason(self, customer_id, query_text):
        # query = ""
        # stored_memory_conversations = self.get_store_memory(customer_id, n=1)
        # if isinstance(stored_memory_conversations, dict):
        #     for key, value in stored_memory_conversations.items():
        #         query = value.get("query_text", "")
        #
        # is_another_order_case = any(phrase in query.lower() for phrase in [
        #     "another order", "different order", "other order", "one more order", "another ticket", "other order",
        #     "different order"
        # ])
        # if is_another_order_case:
        #     query = ''
        # system_prompt = f"""You are an AI assistant that extracts order cancellation reason codes based on user queries.
        #                         You have a predefined mapping of cancellation reasons and their corresponding codes.
        #
        #                         **Mapping:**
        #                         {{
        #                             "001": "Price of one or more items in the order have changed due to which buyer was asked to make additional payment",
        #                             "002": "One or more items in the order not available",
        #                             "003": "Product available at lower than order price",
        #                             "004": "Store is not accepting order",
        #                             "005": "Store rejected the order",
        #                             "006": "Order / fulfillment not received as per buyer app TAT SLA",
        #                             "007": "Order / fulfillment not received as per buyer app TAT SLA",
        #                             "008": "Order / fulfillment not ready for pickup",
        #                             "009": "Wrong product delivered",
        #                             "010": "Buyer wants to modify address / other order details",
        #                             "011": "Buyer not found or cannot be contacted",
        #                             "012": "Buyer does not want product any more",
        #                             "013": "Buyer refused to accept delivery",
        #                             "014": "Delivery address incorrect or not found",
        #                             "015": "Buyer not available at location",
        #                             "016": "Accident / rain / strike / vehicle issues",
        #                             "017": "Order delivery delayed or not possible due to damages, etc.",
        #                             "018": "Delivery address not serviceable",
        #                             "019": "Pickup pin code not serviceable",
        #                             "020": "Order lost in transit",
        #                             "999": "Order confirmation failure",
        #                             "998": "Order confirmation failure"
        #                         }}
        #
        #                         **Task:**
        #                         - Analyze the user query and determine if it matches any cancellation reason from the mapping.
        #                         - If the query contains context that related to valid cancellation reason, return the corresponding cancellation code.
        #                         - If no valid reason is explicitly mentioned, return an empty cancellation code instead of assuming a reason.
        #                         - **Strictly do not assume a cancellation reason if none is found related in the query text.**
        #
        #                         **Example Scenarios:**
        #
        #                         **Example 1:**
        #                         **User Query:** "I want to cancel my order because the product is not available."
        #                         **Expected Output:**
        #                         ```json
        #                         {{
        #                             "cancel_reason_code": "002"
        #                         }}
        #                         ```
        #
        #                         **Example 2:**
        #                         **User Query:** "I want to order another product"
        #                         **Expected Output:**
        #                         ```json
        #                         {{
        #                             "cancel_reason_code": "012"
        #                         }}
        #                         ```
        #                         **Expected Output Format (JSON):**
        #                         ```json
        #                         {{
        #                             "cancel_reason_code": ""
        #                         }}
        #                         ```
        # """

        system_prompt = f"""
        You are a multilingual AI assistant that extracts **order cancellation reason codes** from user queries, which can be in **English, Hindi, Marathi, or a mix like Hinglish**.

        **Objective:**
        Analyze the user query and map it to one of the predefined cancellation reasons (given below). If no clear reason is present in the query, return an empty code.

        **Cancellation Code Mapping:**
        {{
            "001": "Price of one or more items in the order have changed due to which buyer was asked to make additional payment",
            "002": "One or more items in the order not available",
            "003": "Product available at lower than order price",
            "004": "Store is not accepting order",
            "005": "Store rejected the order",
            "006": "Order / fulfillment not received as per buyer app TAT SLA",
            "007": "Order / fulfillment not received as per buyer app TAT SLA",
            "008": "Order / fulfillment not ready for pickup",
            "009": "Wrong product delivered",
            "010": "Buyer wants to modify address / other order details",
            "011": "Buyer not found or cannot be contacted",
            "012": "Buyer does not want product any more",
            "013": "Buyer refused to accept delivery",
            "014": "Delivery address incorrect or not found",
            "015": "Buyer not available at location",
            "016": "Accident / rain / strike / vehicle issues",
            "017": "Order delivery delayed or not possible due to damages, etc.",
            "018": "Delivery address not serviceable",
            "019": "Pickup pin code not serviceable",
            "020": "Order lost in transit",
            "999": "Order confirmation failure",
            "998": "Order confirmation failure"
        }}

        **Instructions:**
        - The query may be in English, Hindi, Marathi, Hinglish, or a combination. Understand the user's intent regardless of the language.
        - Identify if the query matches any valid reason above.
        - If yes, return the corresponding `"cancel_reason_code"`.
        - If not, return:
        ```json
        {{
            "cancel_reason_code": ""
        }}```
        """

        user_prompt = f""" **User Query Details** {query_text}"""
        #  ** Past user query: {query} ** """

        response_payload = {
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            "model_name": "gpt-4o-mini"
        }
        try:
            response1 = model.chat_complete(payload=response_payload)
            response_text = response1.get('answer', "Unable to generate a response.")

            if "```json" in response_text:
                start_index = response_text.find("```json") + 7
                end_index = response_text.rfind("```")
                extracted_json = response_text[start_index:end_index].strip()
            else:
                extracted_json = response_text.strip()

            valid_data = json.loads(extracted_json)
            return valid_data
        except Exception as e:
            return {"cancel_reason_code": ""}

    def cancel_order_api_call(self, customer_id, order_item_id, cancel_reason_id, product_id, session_token):
        customer_id = self.get_customer_id_from_retail_sales(customer_id)
        if not customer_id:
            return "Invalid customer ID"
        product_id = int(product_id) if product_id is not None and str(product_id).isdigit() else 0
        url = f"https://nodeappapi1.plotch.io/api/v1/user/cancelOrderItem?orderItemId={order_item_id}&cancelReasonId={cancel_reason_id}&productId={product_id}"
        # Set headers
        headers = {
            "authorization": f"{session_token}",
            "customerid": str(customer_id)
        }
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            response_data = response.json()
            return response_data.get("m", "No message found"), response.status_code
        except requests.exceptions.RequestException as e:
            return "We couldn't process your cancellation request due to a Technical issue. Please try again later."

    def ecommerce_cancel_intiate(self, order_id, customer_id, query_text, session_token, order_item_id, product_id,
                                 status):
        cancel_order_api = False
        ProductID = product_id
        OrderItemID = order_item_id
        if status == 8:
            return f"Your order {order_id} has already been canceled.would you like to give any other orderID ?", cancel_order_api
        if status == 7:
            return f"Your order {order_id} has already been delivered.would you like to give any other orderID ?", cancel_order_api
        if status in [11, 12, 13, 14, 15, 16]:
            return f"Your order {order_id} is already in the return process with status: {status}. Please wait for the process to complete.", cancel_order_api

        # Fetch cancellation reason code
        cancel_reason_data = self.get_cancel_reason(customer_id, query_text)
        CancelReasonID = cancel_reason_data.get('cancel_reason_code', '')

        # Validate that required IDs are available
        if not CancelReasonID:
            cancel_order_api_response = "Please provide a valid reason for cancel", cancel_order_api

        else:
            cancel_order_api_response, status_code = self.cancel_order_api_call(customer_id, OrderItemID,
                                                                                CancelReasonID, ProductID,
                                                                                session_token)
            if status_code == 200:
                cancel_order_api = True

        return cancel_order_api_response, cancel_order_api

    def order_cancel_intent_detection(self, customer_id, query_text):
        payload = {
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "You are an expert intent classifier for **e-commerce order cancellation queries**. "
                        "Your task is to analyze the query and classify it into one of the predefined e-commerce order intents.\n\n"

                        "**Classification Rules:**\n"
                        "1. Focus **only** on e-commerce order cancellation queries (ignore unrelated topics).\n"
                        "2. Match the **MOST SPECIFIC** intent based on user request.\n"
                        "3. Consider synonyms, paraphrases, and missing details.\n"
                        "4. Ignore minor variations like missing order numbers.\n"
                        "5. Always return **ONLY ONE intent name** from the list below (no extra text).\n\n"

                        "**E-commerce Order Cancellation Intent Taxonomy:**\n\n"

                        "1. **ecommerce_order_cancel** - Queries about normal order cancellation.\n"
                        "- If a user asks about canceling an order without specifically mentioning another order, classify it as ecommerce_order_cancel."
                        "   Examples: 'Can I cancel my order?', 'Please cancel my order #12345.', 'I want to cancel my recent order.'\n\n"

                        "2. **ecommerce_another_order** - Queries about canceling another order or asking about canceling an additional order.\n"
                        "   Examples: 'Can I cancel another order?', 'I need to cancel one more order.', 'Cancel my next order.'मुझे अपना एक और ऑर्डर रद्द करना है।','मला माझा आणखी एक ऑर्डर रद्द करायचा आहे.'હું મારું બીજું ઓર્ડર રદ્દ કરવું છે.'\n\n"

                        "3. **ecommerce_no_intent** - If the query is unclear or unrelated to order cancellations.\n"
                        "   Examples: 'Where is my order?', 'I want to track my delivery.'\n\n"

                        "**Output Rules:**\n"
                        "- Return EXACTLY ONE intent name from the list above.\n"
                        "- Never add explanations or additional text.\n"
                        "- Prioritize the most specific intent based on user input.\n"
                        "- If the query is unclear but order-related, classify it as **ecommerce_no_intent**."
                    )
                },
                {
                    "role": "user",
                    "content": (
                        f"Classify the following query into **one of the intents above**:\n\n"
                        f"Query: '{query_text}'\n\n"
                        "Follow these steps:\n"
                        "1. If the user asks about canceling a **specific order** or uses phrases like 'cancel my order', classify it as **ecommerce_order_cancel**.\n"
                        "2. If the user asks about canceling **another order** or uses phrases like 'cancel one more order' or 'another order', classify it as **ecommerce_another_order**.\n"
                        "3. If the query is unrelated to cancellation, classify it as **ecommerce_no_intent**."
                    )
                }
            ],
            "model_name": "gpt-4o-mini"
        }

        answer = model.chat_complete(payload)
        intent = answer.get('answer')

        valid_intents = [
            "ecommerce_order_cancel",
            "ecommerce_another_order",
            "ecommerce_no_intent"
        ]

        if intent in valid_intents:
            return intent

    def ecommerce_order_cancel_handler(self, payload):
        query_text = payload.get('query_text', '')
        intent = payload.get('intent', '')
        entities = payload.get('entities', [])
        metadata = payload.get('metadata', {})
        message_id = payload.get("message_id", "")
        entity = entities[0]
        customer_id = metadata.get('customer_id', '')
        task_id = payload.get("task_id", "")
        session_token = metadata.get('session_token')

        session_response = self.validate_session(payload)

        session_valid = False
        if session_response.get('metadata'):
            session_valid = session_response['metadata'].get('valid_session', 0) == 1

        if not session_valid:
            return {
                "query_text": query_text,
                "response_text": session_response.get('response_text', 'Session validation failed'),
                "intent": intent,
                "entities": entities,
                "metadata": metadata
            }

        if intent != "ecommerce_order_cancel":
            return {"error": f"Unsupported intent: {intent}"}, 400
        try:
            # rewrite query
            agent_details = {
                "agent_name": "Order Cancellation Assistant",
                "agent_description": "An AI assistant specializing in handling order cancellations by identifying valid cancellation reasons, ensuring accurate processing, and maintaining conversation context.",
                "agent_role": "Analyze user queries and past interactions to extract explicit cancellation reasons, verify order details, and generate structured responses without making assumptions or inferring missing information."
            }

            rewrite_query = self.rewrite_query_text(query_text, customer_id, agent_details, n=4)
            extract_data = {}
            extract_data = {
                "order_id": ""
            }
            # extract entities from query
            query_entities = self.query_entity_extraction(rewrite_query, extract_data)
            detected_intent = self.order_cancel_intent_detection(customer_id, rewrite_query)
            print('detected intent', detected_intent)

            # validate customer
            valid_order_id = self.validate_order_id(str(query_entities.get('order_id')), customer_id)
            response_entities = [{"Order_status": "",
                                  "Order_Id": "",
                                  "Quantity": "",
                                  "Order_Placed_on": "",
                                  "Product_name": "",
                                  "Product_expected_delivery_date": "",
                                  "Product_image": ""}]

            if not valid_order_id:
                metadata['groclake_widget_card_header_text'] = ''
                metadata['groclake_widget_type'] = 'ecommerce_myorders_carousel'
                metadata['groclake_widget_response_followup_text'] = ''
                return {
                    "query_text": query_text,
                    "response_text": "You are not authorized to access this order. Please check your customer ID or contact support for assistance.",
                    "intent": intent,
                    "entities": response_entities,
                    "metadata": metadata
                }

            session_token = metadata.get('session_token')

            response, response_entities, sql_query = self.order_status_response_of_cancel_order_intent(
                str(query_entities.get('order_id')), customer_id, metadata, rewrite_query, detected_intent,
                session_token, query_text)
            response_text = response.get('response_text')

            metadata['groclake_widget_type'] = 'ecommerce_myorders_carousel'
            metadata['groclake_widget_response_followup_text'] = response.get('follow_up_text', '')
            metadata['forceful_intent'] = "ecommerce_order_cancel"

            response_data = {
                "response_text": response_text,
                "intent": "ecommerce_order_cancel",
                "metadata": metadata,
                "query_text": rewrite_query,
                "entities": response_entities
            }
            # send sql query to agentsmith
            agentsmith_payload = {
                "query_text": "log this message",
                "intent": "log",
                "entities": [
                    {
                        "client_agent_uuid": "00a2a625-24b6-4a53-8a38-da49c0cb21b9",
                        "query_text": query_text,
                        "response_text": sql_query,
                        "server_agent_uuid": "string"
                    }
                ],
                "metadata": {
                    "additional_info": "Agent log message request",
                    "nums_offset": "nums_offset",
                    "nums_offset_item": "nums_offset_item",
                }
            }
            agentsmith_uuid = 'e4b77981-4f7e-4d33-9e92-d84cbe0c05cb'
            task_id = 'task_id'

            # Send query to another system
            response_to_log_query = self.adaptor.sendQuery(agentsmith_uuid, agentsmith_payload, task_id)

            if isinstance(response, tuple):
                return response

            # Log the query to another system
            agentsmith_payload = {
                "query_text": "log this message",
                "intent": "log",
                "entities": [
                    {
                        "client_agent_uuid": "00a2a625-24b6-4a53-8a38-da49c0cb21b9",
                        "query_text": rewrite_query,
                        "response_text": response_text,
                        "server_agent_uuid": "string"
                    }
                ],
                "metadata": {
                    "additional_info": "Agent log message request",
                    "nums_offset": "nums_offset",
                    "nums_offset_item": "nums_offset_item",
                }
            }
            agentsmith_uuid = 'e4b77981-4f7e-4d33-9e92-d84cbe0c05cb'
            task_id = 'task_id'

            # Send query to another system
            response_to_log = self.adaptor.sendQuery(agentsmith_uuid, agentsmith_payload, task_id)

            # Task smith
            tasksmith_payload = {
                "query_text": "save this agent's task",
                "intent": "save_task",
                "entities": [
                    {
                        "task_payload": {"query_text": query_text, "resposne_text": response_text}
                    }
                ],
                "metadata": {
                    "source": "user_upload",
                    "handover_timestamp": self.get_current_datetime(),  # current timestamp
                    "status": "completed",
                    "previous_agent": "00a2a625-24b6-4a53-8a38-da49c0cb21b9",  # (client_agent_uuid)
                    "current_agent": ""  # returns the agent uuid.
                }
            }
            tasksmith_uuid = '3bc99771-ba73-466c-88a5-5b6a162f724f'
            response_to_save = self.adaptor.sendQuery(tasksmith_uuid, tasksmith_payload, task_id)

            return response_data

        except Exception as e:
            return {
                "query_text": query_text if 'query_text' in locals() else "",
                "response_text": f"Error: {str(e)}",
                "intent": intent if 'intent' in locals() else "",
                "entities": entities if 'entities' in locals() else [],
                "metadata": metadata if 'metadata' in locals() else {}
            }

    # --- Order status intent  (default handler)-----
    def customer_last_order(self, customer_id):
        customer_id = self.get_customer_id_from_retail_sales(customer_id)
        query = f"SELECT customer_id, order_id ,created_at FROM retail_sales WHERE customer_id = {customer_id} ORDER BY created_at DESC LIMIT 1;"
        result = self.execute_query(query)
        order_id = None
        if result:
            order_id = result[0].get('order_id')
        sql_query = f"""SELECT product_id, name, order_id, status, tracking_url, expected_delivery_date,created_at,qty,cancel_comment,cancelled_at FROM retail_sales_item WHERE order_id = {order_id} LIMIT 1;"""
        execution_result = self.execute_query(sql_query)
        if execution_result:
            return str(order_id), execution_result, sql_query
        else:
            return str(order_id), {'status': 'Order not found'}, sql_query

    def order_status_code(self):
        order_status_codes = {
            0: "Pending",
            1: "Processing",
            2: "Tracking Request In Process",
            3: "Tracking Requested",
            4: "Ready For Pickup",
            5: "Picked Up",
            6: "Dispatched",
            7: "Delivered",
            8: "Cancelled",
            9: "RTO Delivered",
            10: "RTO In Transit",
            11: "Return Requested",
            12: "Return Initiated",
            13: "Ready For Reverse Pickup",
            14: "Return Picked Up",
            15: "Return Dispatched",
            16: "Returned",
            17: "Assume Delivered",
            18: "Lost & Damage",
            19: "Reject",
            20: "Approved",
            21: "Accepted",
            22: "Reverse Tracking Request In Process",
            23: "Reverse Tracking Requested",
            24: "Product Out Of Stock",
            25: "Dispute Raised",
            26: "Out For Delivery",
            27: "Refused By Receiving Party",
            28: "Pending For Confirmation",
            29: "Pending for Cancellation",
            30: "Pending for Return Confirmation",
            31: "Return Liquidated",
            32: "Return Rejected",
            33: "Return Approved",
            34: "Tracking Generated",
            35: "Agent Assigned",
            36: "RTO Disposed",
            37: "Return Pickup Failed",
            38: "Return Failed",
            39: "Out for pickup",
            40: "Pickup failed",
            41: "In Transit",
            42: "At destination hub",
            43: "Delivery failed"
        }
        return order_status_codes

    def order_status_response(self, order_id, customer_id, metadata, query_text, execution_result, intent):
        response_entities_execution_result = ''
        agent_name = "Order Status Assistant"
        agent_description = "an AI assistant specializing in order tracking, order details, delivery updates, and issue resolution."
        agent_role = "assist users with order-related queries, including checking order status, delivery estimates, and resolving issues."

        valid_order_details = self.get_valid_order_id(order_id, customer_id)
        valid_order_id = valid_order_details.get('valid_order_id')

        past_conversation_text = self.get_past_conversation(customer_id, n=2)
        response_entities_execution_result = execution_result
        sql_query = None
        order_status_code = ""

        if not valid_order_id and intent not in ["ecommerce_orderstatus_lastorder", "ecommerce_orderstatus_item",
                                                 "ecommerce_ordertrack_item", 'ecommerce_orderstatus_allorders',
                                                 'ecommerce_all_cancelled_orders', 'ecommerce_all_delivered_orders',
                                                 'ecommerce_all_pending_orders']:
            valid_order_id, execution_result, sql_query = self.customer_last_order(customer_id)
            order_status_note = "The system could not find the specified order, so it retrieved the last known order."
            response_entities_execution_result = execution_result
            order_id = valid_order_id

        if execution_result:
            order_status_code = self.order_status_code().get(execution_result[0].get('status', ''))
            past_conversation_text = ""
            response_entities_execution_result = execution_result
            order_id = str(execution_result[0].get('order_id'))

        if intent in ['ecommerce_orderstatus_item', 'ecommerce_ordertrack_item']:
            valid_order_id = execution_result[0].get('order_id') if execution_result else ''
            execution_result = execution_result
        response_text = ''
        if intent in ['ecommerce_orderstatus_allorders', 'ecommerce_all_cancelled_orders',
                      'ecommerce_all_delivered_orders', 'ecommerce_all_pending_orders']:
            if not response_entities_execution_result or all(
                    not v for v in response_entities_execution_result[0].values()):
                response_text = "It appears there are no orders available at the moment."
            else:
                response_text = "Here are your orders"

        language_prompt = f"""
                   You are a language detection assistant. Detect the natural language of the following query.

                   Instructions:
                   - Return only the name of the detected language.
                   - If the query is written in **pure Hindi (Devanagari script)**, return **Hindi**.
                   - If the query is written in **pure Marathi (Devanagari script)**, return **Marathi**.
                   - If the query is a **mix of Hindi and English written in Roman script**, return **Hinglish**.
                     Examples of such code-mixed text: "mujhe red dress dikhao", "branded shoes chahiye", "mobile ka cover dikhao".

                   Strict Rules:
                   - Do not explain your answer.
                   - Do not provide multiple options.
                   - Only return a single word: the name of the language.

                   Query: "{query_text}"
                   """
        payload = {
            "messages": [
                {"role": "system", "content": language_prompt},
                {"role": "user", "content": query_text}
            ],
            "token_size": 100
        }
        response = model.chat_complete(payload=payload)
        detected_language = response.get("answer", "English").strip()

        system_prompt = f"""
        ## You are an AI-powered assistant named {agent_name}, {agent_description}.
        Your primary role is to {agent_role}, ensuring seamless and intelligent interactions.

        ### Identifying and Handling Queries
        - First, determine if the query is related to an order or is a general/unrelated inquiry.
        - If the user greets (e.g., "Hi", "Hello"), respond: "Hi there! Need help with an order? Let me know!"

        ### Handling General Inquiries
        - If unrelated to orders, respond: "I'm here to help only with order-related queries. Please ask something related to your order."

        ### Handling Order-Related Queries
        - Only respond with info present in the provided MySQL data.
        - Use {order_status_code} to determine order state.
        - Do not assume or fabricate details not present in the data.
        - Always mention the **Order ID**, never mention **Transaction ID**, profile info, or location.

        ### `response_text` Rules
        - Use plain, conversational tone.
        - Do not include AI-like filler (e.g., "Let me check", "As an assistant").
        - Avoid bullet points or markdown.
        - Ensure `response_text` is in the **same language** as the user's 'query_text'
        - If intent is one of ['ecommerce_orderstatus_allorders', 'ecommerce_all_cancelled_orders', 'ecommerce_all_delivered_orders', 'ecommerce_all_pending_orders'], return a general message like:
          - English: "Here are your orders."
          - Hindi: "यह हैं आपके सभी ऑर्डर।"
          - Marathi: "हे आहेत तुमचे सर्व ऑर्डर."
          - Hinglish: "Ye rahe apke sabhi orders".

        ### `follow_up_text` Rules
        - Always include `follow_up_text`.
        - It should briefly guide the next step (e.g., "Do you want to check another order?").
        - Do not assume or ask questions. No chit-chat.

         ### Language Rule
           - Respond in this language: **{detected_language}**.
           - Match the script strictly (e.g., Hinglish uses Roman script, Hindi uses Devanagari).
           - Do not mix languages or scripts.
           - **Important:** Both `response_text` and `follow_up_text` **must be in the same language** as the user 'query_text'.


        ### Output Format
        Return a JSON in the format:
        ```json
        {{
          "response_text": "Main response to the user's query.",
          "follow_up_text": "Helpful next step or guidance."
        }}
        """

        user_prompt = f"""
                       ### **User Query Details**
                       - **Customer Query:** "{query_text}"
                       - **Valid Order ID:** "{order_id or 'None'}"
                       - **Order Status:** {order_status_code}"
                       - **Response Text :**{response_text}
                       - **intent :**{intent}
                       - **MySQL Data (Formatted for Clarity):**{execution_result}
                       - **Past Conversation:** "{past_conversation_text}"
                       """
        response_payload = {
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            "model_name": "gpt-4o-mini"
        }
        response = model.chat_complete(payload=response_payload).get('answer')
        try:

            if "```json" in response:
                start_index = response.find("```json") + 7
                end_index = response.rfind("```")
                extracted_json = response[start_index:end_index].strip()
            else:
                extracted_json = response.strip()
            valid_data = json.loads(extracted_json)
            response_text = valid_data.get('response_text', '')
        except Exception as e:
            response_text = {
                "response_text": response,
                "follow_up_text": "If you need any more information or assistance, feel free to ask!"
            }
            valid_data = response_text
        if order_id and order_id.isdigit() and response:
            context_entities = {"order_id": valid_order_id}
            self.create_memory_for_context_completion(query_text, context_entities, response_text, metadata,
                                                      customer_id)
        response_entities = self.order_response_entities(response_entities_execution_result, intent)
        return valid_data, response_entities, sql_query

    def intent_detection(self, customer_id, query_text):
        payload = {
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "You are an expert intent classifier for **e-commerce order-related queries**. "
                        "Your task is to analyze the query and classify it into one of the predefined e-commerce order intents.\n\n"

                        "**Classification Rules:**\n"
                        "1. Focus **only** on e-commerce order-related queries (ignore unrelated topics).\n"
                        "2. Match the **MOST SPECIFIC** intent based on user request.\n"
                        "3. Consider synonyms, paraphrases, and missing details.\n"
                        "4. Ignore minor variations like missing order numbers.\n"
                        "5. Always return **ONLY ONE intent name** from the list below (no extra text).\n\n"

                        "**E-commerce Order Intent Taxonomy:**\n\n"

                        "1. **ecommerce_orderstatus_lastorder** - Queries about the user's most recent order,latest purchase.\n"
                        "- If a user asks about **last order**, **most recent order**, or **latest purchase** and **does not mention an order ID**, classify it as `ecommerce_orderstatus_lastorder`."
                        "   Examples: 'what is status of my order?','What was my last order?', 'Show me the details of my most recent purchase.'\n\n"

                        "2. **ecommerce_orderstatus_item** - Queries about the items in a specific order.\n"
                        "   Examples: 'What items are in order #12345?', 'Show me the products in my last order', 'Maine jo smartwatch mangayi thi uska status batao.'\n\n"

                        "3. **ecommerce_orderstatus_orderid** - Queries about the status of a specific order using its ID (Use only if order id given in query)\n"
                        "   Examples: 'What is the status of order #12345?', 'Check the status of my order 12345.'\n\n"

                        "4. **ecommerce_ordertrack_orderid** - Queries about tracking a specific order using its ID.\n"
                        "   Examples: 'Where is my order #12345?', 'Track my order with ID 12345.'\n\n"

                        "5. **ecommerce_ordertrack_item** - Queries about tracking specific items in an order.\n"
                        "   Examples: 'Track the items in order #12345.', 'What is the status of the products in my order?'\n\n"

                        "6. **ecommerce_ordertrack_lastorder** - Queries about tracking the user's most recent order.\n"
                        "   Examples: 'Track my last order.', 'Where is my most recent purchase?'\n\n"

                        "7. **ecommerce_ordertrack_phone** - Queries about tracking orders using a phone number.\n"
                        "   Examples: 'Track my order with phone number 9876543210.', 'Find my order using my contact number.'\n\n"

                        "8. **ecommerce_ordertrack_email** - Queries about tracking orders using an email address.\n"
                        "   Examples: 'Track my order with email user@example.com.', 'Find my order using my email.'\n\n"

                        "9. **ecommerce_orderstatus_allorders** - Queries about retrieving all orders of all statuses of a customer.\n"
                        "   Examples: \n"
                        "   - 'Show me all my past orders.'\n"
                        "   - 'List all my orders.'\n"
                        "   - 'Show my order history.'\n"
                        "   - 'Retrieve my last few orders.'\n"
                        "   - 'Show me my most recent 5 orders.'\n"
                        "   - 'Give me a summary of my last 3 purchases.'\n"
                        "   - 'Can I see my purchase history?'\n\n"
                        "   - 'give me status of my 4 orders"

                        "10. **ecommerce_cancel_reason** - Queries about why an order was canceled.\n"
                        "   Examples:\n"
                        "   - 'Why was my order #12345 canceled?'\n"
                        "   - 'Give me the reason for my order cancellation.'\n"
                        "   - 'Can you explain why my order was not processed?'\n"
                        "   - 'I need to know why my purchase was canceled.'\n\n"

                        "11. **ecommerce_all_cancelled_orders** - Queries about retrieving all canceled orders.\n"
                        "   Examples:\n"
                        "   - 'Show me all my canceled orders.'\n"
                        "   - 'List my past canceled purchases.'\n"

                        "12. **ecommerce_all_delivered_orders** - Queries about retrieving all delivered orders.\n"
                        "   Examples:\n"
                        "   - 'Show me all my delivered orders.'\n"
                        "   - 'How many orders have been delivered to me?'\n"
                        "   - 'give me all delivered orders"

                        "13. **ecommerce_all_pending_orders** - Queries about retrieving all pending orders.\n"
                        "   Examples:\n"
                        "   - 'Show me all my pending orders.'\n"
                        "   - 'Which of my orders are still pending?'\n"
                        "   - 'List all orders that are not yet delivered.'\n"
                        "   - 'give me all my pending orders"

                        "**Output Rules:**\n"
                        "- Return EXACTLY ONE intent name from the list above.\n"
                        "- Never add explanations or additional text.\n"
                        "- Prioritize the most specific intent based on user input.\n"
                        "- If the query is unclear but order-related, classify it as **ecommerce_no_intent**."
                    )
                },
                {
                    "role": "user",
                    "content": (
                        f"Classify the following query into **one of the intents above**:\n\n"
                        f"Query: '{query_text}'\n\n"
                        "Follow these steps:\n"
                        "1. Identify key phrases: **'last order', 'all orders', 'delivered orders', 'order status', 'pending orders'**.\n"
                        "2. If 'all' is present, prioritize **bulk order intents** (e.g., `ecommerce_all_delivered_orders`).\n"
                        "3. If an order ID is detected, classify based on **specific order intents** (e.g., `ecommerce_orderstatus_orderid`).\n"
                        "4. If the user asks about tracking, prioritize **tracking intents** over status.\n"
                        "5. Ensure **no better match exists** and return **exactly one intent** in plain text.\n"

                    )
                }
            ],
            "model_name": "gpt-4o-mini"
        }

        answer = model.chat_complete(payload)
        intent = answer.get('answer')

        valid_intents = [
            "ecommerce_orderstatus_orderid",
            "ecommerce_orderstatus_item",
            "ecommerce_orderstatus_lastorder",
            "ecommerce_ordertrack_orderid",
            "ecommerce_ordertrack_item",
            "ecommerce_ordertrack_lastorder",
            "ecommerce_ordertrack_phone",
            "ecommerce_ordertrack_email",
            "ecommerce_orderstatus_allorders",
            "ecommerce_cancel_reason",
            "ecommerce_all_cancelled_orders",
            "ecommerce_all_delivered_orders",
            "ecommerce_all_pending_orders",
            "ecommerce_no_intent"
        ]
        if intent in valid_intents:
            return intent

    def text_to_Sql(self, intent, order_id, customer_id, order_limit, query_text):
        order_id = self.get_valid_order_id(order_id, customer_id).get('valid_order_id')
        if not order_id:
            order_id = 0

        retail_sales_customer_id = self.get_customer_id_from_retail_sales(customer_id)

        item_name = self.extract_item_name_from_query(customer_id, query_text).get('item_name')

        user_sql_queries = {
            "ecommerce_orderstatus_orderid": f"SELECT product_id,name,order_id, status, tracking_url, expected_delivery_date ,created_at,qty FROM retail_sales_item WHERE order_id = {order_id};",

            "ecommerce_orderstatus_item": f"SELECT rsi.product_id, rsi.name,rsi.order_id, rsi.tracking_url, rsi.status, rsi.shipped_date, rsi.expected_delivery_date,rsi.created_at,rsi.qty FROM retail_sales_item rsi JOIN retail_sales rs ON rsi.order_id = rs.order_id WHERE rs.customer_id = {retail_sales_customer_id} and rsi.name LIKE '{item_name}%' ORDER BY rs.created_at DESC LIMIT 1;",

            "ecommerce_orderstatus_lastorder": f"SELECT rsi.product_id, rsi.name,rsi.order_id, rsi.tracking_url, rsi.status, rsi.shipped_date, rsi.expected_delivery_date,rsi.created_at,rsi.qty FROM retail_sales_item rsi JOIN retail_sales rs ON rsi.order_id = rs.order_id WHERE rs.customer_id = {retail_sales_customer_id} ORDER BY rs.created_at DESC LIMIT 1;",

            "ecommerce_ordertrack_orderid": f"SELECT  product_id, name,order_id, tracking_url, status, shipped_date, expected_delivery_date,created_at,qty FROM retail_sales_item WHERE order_id = {order_id};",

            "ecommerce_ordertrack_item": f"SELECT rsi.product_id, rsi.name,rsi.order_id, rsi.tracking_url, rsi.status, rsi.shipped_date, rsi.expected_delivery_date,rsi.created_at,rsi.qty FROM retail_sales_item rsi JOIN retail_sales rs ON rsi.order_id = rs.order_id WHERE rs.customer_id = {retail_sales_customer_id} and rsi.name LIKE '{item_name}%' ORDER BY rs.created_at DESC LIMIT 1;",

            "ecommerce_ordertrack_lastorder": f"SELECT rsi.product_id, rsi.name, rsi.order_id, rsi.tracking_url, rsi.status, rsi.shipped_date, rsi.expected_delivery_date, rsi.created_at, rsi.qty FROM retail_sales_item rsi JOIN retail_sales rs ON rsi.order_id = {order_id} WHERE rs.customer_id = 881140 ORDER BY rs.created_at DESC LIMIT 1;",

            "ecommerce_orderstatus_allorders": f"SELECT rsi.product_id, rsi.name,rsi.order_id, rsi.tracking_url, rsi.status, rsi.shipped_date, rsi.expected_delivery_date,rsi.created_at,rsi.qty FROM retail_sales_item rsi JOIN retail_sales rs ON rsi.order_id = rs.order_id WHERE rs.customer_id = {retail_sales_customer_id} ORDER BY rs.created_at DESC LIMIT {order_limit};",

            "ecommerce_cancel_reason": f"SELECT product_id, name, qty , status ,created_at,order_id, cancel_comment,cancelled_at FROM retail_sales_item WHERE order_id = {order_id} AND status = 8;",

            "ecommerce_all_cancelled_orders": f"SELECT rsi.product_id, rsi.name,rsi.order_id, rsi.tracking_url, rsi.status, rsi.shipped_date, rsi.expected_delivery_date,rsi.created_at,rsi.qty FROM retail_sales_item rsi JOIN retail_sales rs ON rsi.order_id = rs.order_id WHERE rs.customer_id = {retail_sales_customer_id} and rsi.status =8 ORDER BY rs.created_at DESC LIMIT {order_limit};",

            "ecommerce_all_delivered_orders": f"SELECT rsi.product_id, rsi.name,rsi.order_id, rsi.tracking_url, rsi.status, rsi.shipped_date, rsi.expected_delivery_date,rsi.created_at,rsi.qty FROM retail_sales_item rsi JOIN retail_sales rs ON rsi.order_id = rs.order_id WHERE rs.customer_id = {retail_sales_customer_id} and rsi.status =7 ORDER BY rs.created_at DESC LIMIT {order_limit};",

            "ecommerce_all_pending_orders": f"SELECT rsi.product_id, rsi.name,rsi.order_id, rsi.tracking_url, rsi.status, rsi.shipped_date, rsi.expected_delivery_date,rsi.created_at,rsi.qty FROM retail_sales_item rsi JOIN retail_sales rs ON rsi.order_id = rs.order_id WHERE rs.customer_id = {retail_sales_customer_id} and rsi.status IN (0,1) ORDER BY rs.created_at DESC LIMIT {order_limit};"
        }
        if intent:
            return user_sql_queries.get(intent, {})
        return {}

    def default_handler(self, payload):
        query_text = payload.get('query_text', '')
        request_intent = payload.get('intent', '')
        entities = payload.get('entities', [])
        metadata = payload.get('metadata', {})
        message_id = payload.get("message_id", "")
        entity = entities[0]
        customer_id = metadata.get('customer_id', '')
        task_id = payload.get("task_id", "")

        session_response = self.validate_session(payload)

        session_valid = False
        if session_response.get('metadata'):
            session_valid = session_response['metadata'].get('valid_session', 0) == 1

        if not session_valid:
            return {
                "query_text": query_text,
                "response_text": session_response.get('response_text', 'Session validation failed'),
                "intent": request_intent,
                "entities": entities,
                "metadata": metadata
            }

        if request_intent != "ecommerce_order_status":
            return {"error": f"Unsupported intent: {request_intent}"}, 400
        try:

            # detect intent of query text
            intent = self.intent_detection(customer_id, query_text)
            extract_data = {}
            order_limit = 5

            # If inetnt orderstatus_allorders
            if intent in ['ecommerce_orderstatus_allorders', 'ecommerce_all_cancelled_orders',
                          'ecommerce_all_delivered_orders', 'ecommerce_all_pending_orders']:
                extract_data = {
                    'order_id': "",
                    'number_of_orders': ""
                }
            else:
                extract_data = {
                    "order_id": ""
                }

            # extract entities from query
            query_entities = self.query_entity_extraction(query_text, extract_data)

            # set default order limit
            if query_entities.get('number_of_orders'):
                order_limit = query_entities.get('number_of_orders') if query_entities.get(
                    'number_of_orders').isdigit() else 5
                query_entities['order_id'] = ''

            # validate customer
            valid_order_id = self.validate_order_id(str(query_entities.get('order_id')), customer_id)
            response_entities = [{"Order_status": "",
                                  "Order_Id": "",
                                  "Quantity": "",
                                  "Order_Placed_on": "",
                                  "Product_name": "",
                                  "Product_expected_delivery_date": "",
                                  "Product_image": ""}]

            if not valid_order_id:
                metadata['groclake_widget_card_header_text'] = ''
                metadata['groclake_widget_type'] = 'ecommerce_myorders_carousel'
                metadata['groclake_widget_response_followup_text'] = ''
                return {
                    "query_text": query_text,
                    "response_text": "You are not authorized to access this order. Please check your customer ID or contact support for assistance.",
                    "intent": intent,
                    "entities": response_entities,
                    "metadata": metadata
                }
            # rewrite query
            agent_details = {
                "agent_name": "Order Status Assistant",
                "agent_description": "An AI assistant specializing in providing real-time order status updates, tracking deliveries, and addressing user concerns related to order fulfillment.",
                "agent_role": "Analyze user queries and past interactions to accurately retrieve order status, estimated delivery times, and tracking details, ensuring clear and precise responses without assumptions."
            }

            rewrite_query = self.rewrite_query_text(query_text, customer_id, agent_details, n=1)

            # text to sql
            new_sql_query = self.text_to_Sql(intent, query_entities.get('order_id'), customer_id, order_limit,
                                             rewrite_query)
            # execute sql query
            execution_result = self.execute_query(new_sql_query)
            # sql to text
            response, response_entities, sql_query = self.order_status_response(str(query_entities.get('order_id')),
                                                                                customer_id, metadata, query_text,
                                                                                execution_result, intent)
            # store response in memory
            self.create_memory(query_text, response, intent, entities, customer_id, metadata)
            response_text = response.get('response_text')


            if intent == 'ecommerce_cancel_reason':
                metadata['groclake_widget_type'] = 'ecommerce_product_carousel'
                metadata[
                    'groclake_widget_response_followup_text'] = "Your order has been canceled. Here are some recommendations for similar items you might like."
            else:
                metadata['groclake_widget_type'] = 'ecommerce_myorders_carousel'
                metadata['groclake_widget_response_followup_text'] = response.get('follow_up_text', '')

            response_data = {
                "response_text": response_text,
                "intent": "ecommerce_order_status",
                "metadata": metadata,
                "query_text": query_text,
                "entities": response_entities
            }
            # send sql query to agentsmith
            agentsmith_payload = {
                "query_text": "log this message",
                "intent": "log",
                "entities": [
                    {
                        "client_agent_uuid": "00a2a625-24b6-4a53-8a38-da49c0cb21b9",
                        "query_text": query_text,
                        "response_text": sql_query if sql_query else new_sql_query,
                        "server_agent_uuid": "string"
                    }
                ],
                "metadata": {
                    "additional_info": "Agent log message request",
                    "nums_offset": "nums_offset",
                    "nums_offset_item": "nums_offset_item",
                }
            }
            agentsmith_uuid = 'e4b77981-4f7e-4d33-9e92-d84cbe0c05cb'
            task_id = 'task_id'

            # Send query to another system
            response_to_log_query = self.adaptor.sendQuery(agentsmith_uuid, agentsmith_payload, task_id)

            if isinstance(response, tuple):
                return response

            # Log the query to another system
            agentsmith_payload = {
                "query_text": "log this message",
                "intent": "log",
                "entities": [
                    {
                        "client_agent_uuid": "00a2a625-24b6-4a53-8a38-da49c0cb21b9",
                        "query_text": query_text,
                        "response_text": response_text,
                        "server_agent_uuid": "string"
                    }
                ],
                "metadata": {
                    "additional_info": "Agent log message request",
                    "nums_offset": "nums_offset",
                    "nums_offset_item": "nums_offset_item",
                }
            }

            agentsmith_uuid = 'e4b77981-4f7e-4d33-9e92-d84cbe0c05cb'
            task_id = 'task_id'

            # Send query to another system
            response_to_log = self.adaptor.sendQuery(agentsmith_uuid, agentsmith_payload, task_id)

            # Task smith
            tasksmith_payload = {
                "query_text": "save this agent's task",
                "intent": "save_task",
                "entities": [
                    {
                        "task_payload": {"query_text": query_text, "resposne_text": response_text}
                    }
                ],
                "metadata": {
                    "source": "user_upload",
                    "handover_timestamp": self.get_current_datetime(),  # current timestamp
                    "status": "completed",
                    "previous_agent": "00a2a625-24b6-4a53-8a38-da49c0cb21b9",  # (client_agent_uuid)
                    "current_agent": ""  # returns the agent uuid.
                }
            }
            tasksmith_uuid = '3bc99771-ba73-466c-88a5-5b6a162f724f'
            response_to_save = self.adaptor.sendQuery(tasksmith_uuid, tasksmith_payload, task_id)

            return response_data

        except Exception as e:
            return {
                "query_text": query_text if 'query_text' in locals() else "",
                "response_text": f"Error: {str(e)}",
                "intent": request_intent if 'intent' in locals() else "",
                "entities": entities if 'entities' in locals() else [],
                "metadata": metadata if 'metadata' in locals() else {}
            }


adaptor_config = {
    'apc_id': 'apc_id',
    'client_agent_uuid': 'Client-Agent-UUID'
}

order_agent = CraftsvillaOrderAgent(app, "orderagent", "ecommerce_order_status", "order status query handle",
                                    adaptor_config)

if __name__ == "__main__":
    order_agent.run(host="0.0.0.0", port=30004, debug=True)

Config.py

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv(dotenv_path="./.env")


class Config:
    MYSQL_CONFIG = {
        'user': os.getenv('MYSQL_USER'),
        'passwd': os.getenv('MYSQL_PASSWORD'),
        'host': os.getenv('MYSQL_HOST'),
        'port': int(os.getenv('MYSQL_PORT')),
        'db': os.getenv('MYSQL_DATABASE'),
        'charset': 'utf8'
    }

    REDIS_CONFIG = {
        'host': os.getenv('REDIS_HOST'),
        'port': os.getenv('REDIS_PORT'),
        'password': None
    }

    CUSTOMER_INSTANCE = os.getenv('CUSTOMER_INSTANCE')

